In [2]:
import torch
from torch import nn
from d2l import torch as d2l

#实现输入x与核矩阵k的互相关运算
def corr2d(X,K):
    h,w=K.shape
    Y=torch.zeros((X.shape[0]-h+1,X.shape[1]-w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j]=(X[i:i+h,j:j+w]*K).sum()
    return Y

#verification
X=torch.arange(9).reshape(3,3)
K=torch.tensor([[0,1],[2,3]])
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

In [ ]:
#自己实现一个卷积层
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight=nn.parameter(torch.rand(kernel_size))
        self.bias=nn.parameter(torch.zeros(1))
    def forward(self,x):
        return corr2d(x,self.weight)+self.bias
#使用nn内置的卷积层，手动完成一个学习卷积核的过程
X=torch.ones((6,8))
X[:,2:6]=0
K=torch.tensor([[1,-1]])
Y=corr2d(X,K)
X,Y
    

(tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
         [1., 1., 0., 0., 0., 0., 1., 1.],
         [1., 1., 0., 0., 0., 0., 1., 1.],
         [1., 1., 0., 0., 0., 0., 1., 1.],
         [1., 1., 0., 0., 0., 0., 1., 1.],
         [1., 1., 0., 0., 0., 0., 1., 1.]]),
 tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.]]))

In [6]:
con_net=nn.Conv2d(1,1,kernel_size=(1,2),bias=False)
X=X.reshape((1,1,6,8))
Y=Y.reshape((1,1,6,7))
lr=3e-2
for i in range(10):
    Y_hat=con_net(X)
    l=(Y_hat-Y)**2
    con_net.zero_grad()
    l.sum().backward()
    con_net.weight.data[:]-=lr*con_net.weight.grad
    if(i+1)%2==0:
        print(f'epoch {i+1},loss {l.sum():.3f}')

epoch 2,loss 11.105
epoch 4,loss 2.818
epoch 6,loss 0.864
epoch 8,loss 0.305
epoch 10,loss 0.117


In [ ]:
con_net.weight.data.reshape((1,2))
#如果不加reshape的话，为什么嵌套层数那么多？

tensor([[ 0.9517, -1.0205]])

In [13]:
#多输入通道互相关
def corr2d_multi_in(X,K):
    return sum(d2l.corr2d(x,k) for x,k in zip(X,K))#无非是每个通道依次和对应的输入矩阵进行互相关运算

X1=torch.tensor([[[0,1,2],[3,4,5],[6,7,8]],[[1,2,3],[4,5,6],[7,8,9]]])
K1=torch.tensor([[[0,1],[2,3]],[[1,2],[3,4]]])
corr2d_multi_in(X1,K1)

tensor([[ 56.,  72.],
        [104., 120.]])

In [16]:
#多输出通道互相关
def corr2d_multi_in_out(X,K):#K此时是一个4维的，最外面的维度是Cout
    return torch.stack([corr2d_multi_in(X,k) for k in K],0) #torch.stack用于将多个输出通道的结果堆叠起来。
K2=torch.stack((K1,K1+1,K1+2),0) #K2.size为3*2*2*2,因为K1.size为2*2*2
corr2d_multi_in_out(X1,K2)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [ ]:
#pooling layer的前向传播
def pool2d(X,pool_size,mode='max'):
    p_h,p_w=pool_size
    Y=torch.zeros((X.shape[0]-p_h+1,X.shape[1]-p_w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode=='max':
                Y[i,j]=X[i:i+p_h,j:j+p_w].max()
            elif mode=='avg':
                Y[i,j]=X[i:i+p_h,j:j+p_w].mean()
    return Y
    

In [ ]:
a = torch.tensor([[1, 2, 3], [4, 5, 6]])
b = torch.tensor([[7, 8, 9], [10, 11, 12]])
c = torch.tensor([[13, 14, 15], [16, 17, 18]])
result1 = torch.cat([a, b, c], dim=0) #6*3!!第 0 维度的大小是各个输入张量在第 0 维度上的大小之和
print(result1)
result2 = torch.cat([a, b, c], dim=1)#横着拼，增加列数
print(result2)

tensor([[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12],
        [13, 14, 15],
        [16, 17, 18]])
tensor([[ 1,  2,  3,  7,  8,  9, 13, 14, 15],
        [ 4,  5,  6, 10, 11, 12, 16, 17, 18]])
